# MSDS 432, Assignment 4 - Quick Sort

#### Author: Brandon Moretz

In [2]:
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from time import time
import matplotlib.style as style

In [3]:
np.random.seed(323) # static seed so results are reproducible

In [4]:
style.use('seaborn-poster') # sets the size of the charts
style.use('ggplot') # set the theme for matplotlib

### Person

In [5]:
class Person:
    
    def __init__(self, first_name, last_name, address):
        """Initialize a new Person object."""
        self._fname = first_name
        self._lname = last_name
        self._address = address
        
    def first_name(self):
        return self._fname
    
    def last_name(self):
        return self._lname
    
    def address(self):
        return self._address


## Sorting Algorithms

In [8]:
def quickSort(array):
    """Quicksort, uses divide and conquer strategy to sort an array, O(n log n)"""
    if len(array) < 2:
        # base case, arrays with 0 or 1 elements are already sorted.
        return array
    else:
        # recursive case
        pivot = array[0]
        # sub-array of all elements less than the pivot
        less = [i for i in array[1:] if i <= pivot]
        # sub-array of all elements greather than the pivot
        greater = [i for i in array[1:] if i >= pivot]
        return quicksort(less) + [pivot] + quicksort(greater)
    
def findSmallest(arr):
    """Finds the smallest element in an array by returning the corresponding index."""
    smallest = arr[0] # Stores the smallest value
    smallest_index = 0 # Stores the smallest index
    
    for i in range(1, len(arr)):
        if arr[i] < smallest:
            smallest = arr[i]
            smallest_index = i
    return smallest_index

def selectionSort(arr):
    """Selection Sort Algorithm, O(N**2)"""
    newArr = []
    for i in range(len(arr)):
        smallest = findSmallest(arr)
        newArr.append(arr.pop(smallest))
    return newArr

def insertionSort(arr):
    """Insertion Sort Algorithm, O(N**2)"""

    for i in range(len(arr)):
        j = i
        while j > 0 and (arr[j] <arr[j-1]):
            arr[j], arr[j-1] = arr[j-1], arr[j]
            j = j - 1
    return arr

def profile(func):
    """Utility function to help profile the runtime of code blocks."""
    start_time = time()
    func()
    return time() - start_time

## Benchmark

In [10]:
print(quicksort([10, 5, 2, 3]))

[2, 3, 5, 10]


## Results

## Summary